In [ ]:
from z3 import *
from itertools import combinations
import time
import numpy as np


# Aux functions

# start_times = x and y coords
# durations and resources = x and y dims
# total = width / height
def cumulative(start_times, durations, resources, total):
    cumulative_expression = []
    for resource in resources:
        cumulative_expression.append(
            sum([
                If(And(start_times[s] <= resource, resource < start_times[s] + durations[s]), 
                resources[s], 
                0) 
                for s in range(len(start_times))
            ]) <= total
        )
    #print(cumulative_expression)
    return cumulative_expression

def cumulative2(opt, s, d, r, c, time):
    for t in range(time):
        tmp_sum = 0
        for i in range(len(s)):
            tmp_sum += If(And(s[i] <= t, t < s[i] + d[i]), 1, 0) * r[i]
        opt.add(c >= tmp_sum) 
    return opt

def maximum(array):
    maximum = array[0]
    for value in array[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum

# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions

# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance


# Solve instance: 
def solve(instance):
    optimizer = Optimize()
    
    timeout = 1 * 1000
    optimizer.set("timeout", timeout)

    # Variable Initalization

    x_dims=[]
    y_dims=[]
    x_coords = []
    y_coords = []

    for x_dim,y_dim in instance.dimensions:
        x_dims.append(x_dim)
        y_dims.append(y_dim)
    
    for block in range(instance.n):
        x_coords.append(Int("x%s" % str(block+1)))
        y_coords.append(Int("y%s" % str(block+1)))

    max_height = math.ceil(sum(y_dims)/(instance.width//max(x_dims)))

    print("Width of the plate: ")
    print(instance.width)
    print("Max height of the plate: ")
    print(max_height)

    # Bounding the domain
    
    optimizer.add([x_coords[block] >= 0 for block in range(instance.n)]) # x domain
    optimizer.add([y_coords[block] >= 0 for block in range(instance.n)]) # y domain
        
    optimizer.add([maximum([x_coords[block] + x_dims[block] for block in range(instance.n)]) <= instance.width]) # fits width
    optimizer.add([maximum([y_coords[block] + y_dims[block] for block in range(instance.n)]) <= max_height]) # fits height
    
    # All Different and Cummulative constraints
    
    optimizer.add([Distinct([1000*x_coords[block]+y_coords[block]]) for block in range(instance.n)])
    #optimizer.add(cumulative(y_coords,y_dims,x_dims,instance.width))
    #optimizer.add(cumulative(x_coords,x_dims,y_dims,max_height))
    optimizer = cumulative2(optimizer, x_coords, x_dims, y_dims, max_height, instance.width)
    optimizer = cumulative2(optimizer, y_coords, y_dims, x_dims, instance.width, max_height)
    
    # No overlapping
    
    for (block1,block2) in combinations(range(instance.n),2):
        optimizer.add(Or(x_coords[block1]+x_dims[block1] <= x_coords[block2],
                        x_coords[block2]+x_dims[block2] <= x_coords[block1],
                        y_coords[block1]+y_dims[block1] <= y_coords[block2],
                        y_coords[block2]+y_dims[block2] <= y_coords[block1]))
    
    
    # Goal: minimize height
    
    real_height = maximum([y_coords[block] + y_dims[block] for block in range(instance.n)])
    
    optimizer.minimize(real_height)

    timeout = 500 * 1000
    
    if optimizer.check() == sat:
        model = optimizer.model()
        solution_height = model.evaluate(real_height).as_string()
        print("Satisfiable")
        print("Solution - Minimum height:")
        print(solution_height)
    else:
        print("Not satisfiable")
        
    return optimizer

def main():
    instance_file = "../instances/ins-12.txt"
    instance = read_file(instance_file)

    start = time.time()
    optimizer = solve(instance)
    end = time.time()

    print("{:.2f}".format(end - start) + " seconds")
    
    #IF YOU ONLY WANT TO RUN 12 look at the bottom code
    
    #for x in [19]:
    for x in range(1,19):
        
        instance_file = "../instances/ins-{}.txt".format(x)
        instance = read_file(instance_file)
        
        start = time.time()
        optimizer = solve(instance)
        end = time.time()
        
        print("{:.2f}".format(end - start) + " seconds")
        
        if optimizer.check() == sat:
            with open('ins{}SMT.txt'.format(x), 'w') as myfile:
                myfile.write('time='+'{:.3f}'.format(end - start))
                myfile.close()
        else:
            with open('ins{}SMT.txt'.format(x), 'w') as myfile:
                myfile.write("time=0")
                myfile.close()
        
    



if __name__ == '__main__':
    main()

Width of the plate: 
19
Max height of the plate: 
29
Not satisfiable
9.50 seconds
Width of the plate: 
8
Max height of the plate: 
16
Satisfiable
Solution - Minimum height:
8
2.08 seconds
Width of the plate: 
9
Max height of the plate: 
9
Satisfiable
Solution - Minimum height:
9
1.48 seconds
Width of the plate: 
10
Max height of the plate: 
15
Satisfiable
Solution - Minimum height:
10
2.59 seconds
Width of the plate: 
11
Max height of the plate: 
29
Not satisfiable
3.83 seconds
Width of the plate: 
12
Max height of the plate: 
23
Satisfiable
Solution - Minimum height:
12
1.82 seconds
Width of the plate: 
13
Max height of the plate: 
46
Not satisfiable
2.81 seconds


In [2]:
stat = "time="

times = []

for x in range(1,21):
    f = open('ins{}SMT.txt'.format(x),'r')
    txt = f.read()
    start = txt.find(stat)
    newtxt = txt[start:]
    end = newtxt.find(stat)
    times.append(float(newtxt[len(stat):end]))
    print(str(x)+",",newtxt[len(stat):end])

SyntaxError: invalid syntax (<ipython-input-2-fa5589f02bfe>, line 5)